In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%cd
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.5-bin-hadoop2.7.tgz > /dev/null
!pip install -q findspark
import os 
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/root/spark-2.4.5-bin-hadoop2.7"
%cd /content
import findspark
findspark.init()

import pyspark
# get a spark context
sc = pyspark.SparkContext.getOrCreate()
print(sc)
# get the context
spark = pyspark.sql.SparkSession.builder.getOrCreate()
print(spark) 

/root
/content
<SparkContext master=local[*] appName=pyspark-shell>


In [ ]:
%cd /content/drive/My Drive/BigData2020/data/lingspam_public 

!cat readme.txt


/content/drive/My Drive/BigData2020/data/lingspam_public
This directory contains the Ling-Spam corpus, as described in the 
paper:

I. Androutsopoulos, J. Koutsias, K.V. Chandrinos, George Paliouras, 
and C.D. Spyropoulos, "An Evaluation of Naive Bayesian Anti-Spam 
Filtering". In Potamias, G., Moustakis, V. and van Someren, M. (Eds.), 
Proceedings of the Workshop on Machine Learning in the New Information 
Age, 11th European Conference on Machine Learning (ECML 2000), 
Barcelona, Spain, pp. 9-17, 2000.

There are four subdirectories, corresponding to four versions of 
the corpus:

bare: Lemmatiser disabled, stop-list disabled.
lemm: Lemmatiser enabled, stop-list disabled.
lemm_stop: Lemmatiser enabled, stop-list enabled.
stop: Lemmatiser disabled, stop-list enabled.

Each one of these 4 directories contains 10 subdirectories (part1, 
..., part10). These correspond to the 10 partitions of the corpus 
that were used in the 10-fold experiments. In each repetition, one 
part was reserved 

## Read the dataset and create RDDs 


In [ ]:
from pathlib import Path
import re

def makeTestTrainRDDs(pathString):
    """ Takes one of the four subdirectories of the lingspam dataset and returns two RDDs one each for testing and training. """
    
    p = Path(pathString) # gets a path object representing the current directory path.
    dirs = list(p.iterdir()) # get the directories part1 ... part10. 
    print(dirs) # Print to check that you have the right directory. You can comment this out when checked. 
    rddList = [] # create a list for the RDDs
    # now create an RDD for each 'part' directory and add them to rddList
    print('creating RDDs')
    for d in dirs: # iterate through the directories
        dir_path = str(d.resolve())
        print(dir_path)
        rdd = sc.wholeTextFiles(dir_path) #>>> # read the files in the directory 
        rddList.append(rdd) #>>> append the RDD to the rddList
    print('len(rddList)', len(rddList))  # we should now have 10 RDDs in the list # just for testing
    print(rddList[1].take(1)) # just for testing, comment out when it works.

    testRDD1 = rddList[9] # set the test set
    trainRDD1 = rddList[0] # start the training set from 0 and 
    # now loop over the range from 1 to 9 (exclusive) to create a union of the remaining RDDs
    print('creating RDD union')
    for i in range(1, 9):
        trainRDD1 = trainRDD1.union(rddList[i]) #>>> create a union of the current and the next 
            # RDD in the list, so that in the end have a union of all parts 0-8. (9 is used as test set)
    # both RDDs should remove the paths and extensions from the filename. 
    testRDD2 = testRDD1.map(lambda fn_txt: (re.split('[/\.]', fn_txt[0])[-2], fn_txt[1])) 
    trainRDD2 = trainRDD1.map(lambda fn_txt: (re.split('[/\.]', fn_txt[0])[-2], fn_txt[1])) 
    return (trainRDD2, testRDD2)


%cd /content/drive/My Drive/BigData2020/data/lingspam_public 

!ls 
# the code below is for testing the function makeTestTrainRDDs
trainRDD_testRDD = makeTestTrainRDDs('bare') # read from the directory 
(trainRDD, testRDD) = trainRDD_testRDD # unpack the returned tuple
print('created the RDDs') # notify the user, so that be able to figure out where things went wrong if they do.
print('testRDD.count(): ', testRDD.count()) # should be ~289
#print('trainRDD.count(): ', trainRDD.count()) # should be ~2604 - commented out to save time as it takes some time to create RDD from all the files
print('testRDD.getNumPartitions():', testRDD.getNumPartitions()) # normally 2 on Colab (single machine)
print('testRDD.getStorageLevel():', testRDD.getStorageLevel()) # Serialized, 1x Replicated, expected to be (False, False, False, False, 1) 
print('testRDD.take(1): ', testRDD.take(1)) # should be (filename, message) 
rdd1 = testRDD # use this for development in the next tasks 

/content/drive/My Drive/BigData2020/data/lingspam_public
bare  lemm  lemm_stop  readme.txt  stop
[PosixPath('bare/part10'), PosixPath('bare/part9'), PosixPath('bare/part8'), PosixPath('bare/part7'), PosixPath('bare/part6'), PosixPath('bare/part5'), PosixPath('bare/part4'), PosixPath('bare/part3'), PosixPath('bare/part2'), PosixPath('bare/part1')]
creating RDDs
/content/drive/My Drive/BigData2020/data/lingspam_public/bare/part10
/content/drive/My Drive/BigData2020/data/lingspam_public/bare/part9
/content/drive/My Drive/BigData2020/data/lingspam_public/bare/part8
/content/drive/My Drive/BigData2020/data/lingspam_public/bare/part7
/content/drive/My Drive/BigData2020/data/lingspam_public/bare/part6
/content/drive/My Drive/BigData2020/data/lingspam_public/bare/part5
/content/drive/My Drive/BigData2020/data/lingspam_public/bare/part4
/content/drive/My Drive/BigData2020/data/lingspam_public/bare/part3
/content/drive/My Drive/BigData2020/data/lingspam_public/bare/part2
/content/drive/My Drive/

## Tokenize and remove punctuation


In [ ]:
import nltk
import re
from nltk.corpus import stopwords

def tokenize(text):
    """ Apply the nltk.word_tokenize() method to our text, return the token list. """
    nltk.download('punkt') #  loads the standard NLTK tokenizer model 
    # it is important that this is done here in the function, as it needs to be done on every worker.
    # If do the download outside this function, it would only be executed on the driver     
    return (nltk.word_tokenize(text)) # use the nltk function word_tokenize
    
def removePunctuation(tokens):
    """ Remove punctuation characters from all tokens in a provided list. """
   
    tokens2 =  [re.sub('[()\[\],.?!";_]','', s) for s in tokens] # used a list comprehension to remove punctuaton
    return tokens2
    
def prepareTokenRDD(fn_txt_RDD):
    """ Take an RDD with (filename, text) elements and transform it into a (filename, [token ...]) RDD without punctuation characters. """
    rdd_vals2 = fn_txt_RDD.values() # It's convenient to process only the values. 
    rdd_vals3 = rdd_vals2.map(tokenize) # Create a tokenised version of the values by mapping
    rdd_vals4 = rdd_vals3.map(removePunctuation) # remove punctuation from the values
    rdd_kv = fn_txt_RDD.keys().zip(rdd_vals4) # zip the two RDDs together 
    # i.e. produce tuples with one item from each RDD.
    # This works because have only applied mappings to the values, 
    # therefore the items in both RDDs are still aligned.
    # now remove any empty strings (i.e. length 0) 
    # created by removing punctuation, and resulting entries without words left.
    rdd_kvr = rdd_kv.map(lambda v: (v[0], [e for e in v[1] if len(e)>0])) # remove empty strings from token lists using RDD.map and a lambda. TIP len(s) gives you the lenght of string. 
    rdd_kvrf = rdd_kvr.filter(lambda v: [len(v[1])>0]) # remove empty token lists using RDD.filter and a lambda. 
    
    return rdd_kvrf 

rdd2 = prepareTokenRDD(rdd1) # Use a small RDD for testing.
print(rdd2.take(1)) # For checking result of task 2. 

[('3-1msg1', ['Subject', ':', 're', ':', '2', '882', 's', '-', '>', 'np', 'np', '>', 'date', ':', 'sun', '15', 'dec', '91', '02', ':', '25', ':', '02', 'est', '>', 'from', ':', 'michael', '<', 'mmorse', '@', 'vm1', 'yorku', 'ca', '>', '>', 'subject', ':', 're', ':', '2', '864', 'queries', '>', '>', 'wlodek', 'zadrozny', 'asks', 'if', 'there', 'is', '``', 'anything', 'interesting', '``', 'to', 'be', 'said', '>', 'about', 'the', 'construction', '``', 's', '>', 'np', 'np', '``', 'second', '>', 'and', 'very', 'much', 'related', ':', 'might', 'we', 'consider', 'the', 'construction', 'to', 'be', 'a', 'form', '>', 'of', 'what', 'has', 'been', 'discussed', 'on', 'this', 'list', 'of', 'late', 'as', 'reduplication', 'the', '>', 'logical', 'sense', 'of', '``', 'john', 'mcnamara', 'the', 'name', '``', 'is', 'tautologous', 'and', 'thus', 'at', '>', 'that', 'level', 'indistinguishable', 'from', '``', 'well', 'well', 'now', 'what', 'have', 'we', 'here', '``', 'to', 'say', 'that', "'", 'john', 'mcnama

##  Creating normalised TF.IDF vectors of defined dimensionality.



In [ ]:
# use the hashing trick to create a fixed-size vector from a word list
def hashing_vectorize(text, N): # arguments: the list and the size of the output vector
    v = [0] * N  # create vector of 0s
    for word in text: # iterate through the words 
      h = hash(word)  # get the hash value 
      v[h % N] = v[h % N] + 1  # add 1 at the hashed address
    return v # return hashed word vector

from pyspark.mllib.feature import IDF, Normalizer

def normTFIDF(fn_tokens_RDD, vecDim):
    keysRDD = fn_tokens_RDD.keys()
    tokensRDD = fn_tokens_RDD.values()
    tfVecRDD = tokensRDD.map(lambda tokens: hashing_vectorize(tokens, vecDim)) 
    idf = IDF() # create IDF object
    idfModel = idf.fit(tfVecRDD) # calculate IDF values
    tfIdfRDD = idfModel.transform(tfVecRDD) # 2nd pass needed (see lecture slides), transforms RDD
    norm = Normalizer(1) # create a Normalizer object like in the example linked above
    normTfIdfRDD = norm.transform(tfIdfRDD) # and apply it to the tfIdfRDD 
    zippedRDD = keysRDD.zip(normTfIdfRDD) # zip the keys and values together
    return zippedRDD

testDim = 10 # too small for good accuracy, but OK for testing
rdd3 = normTFIDF(rdd2, testDim) # test normTFIDF function
print(rdd3.take(1)) # now have tuples with ('filename', [N-dim vector])
# e.g. [('3-1msg1', DenseVector([0.0, 0.1629, 0.6826, 0.0, 0.0, 0.0, 0.4017, 0.3258, 0.3133, 0.3766]))]

[('3-1msg1', DenseVector([0.0, 0.072, 0.3017, 0.0, 0.0, 0.0, 0.1775, 0.144, 0.1384, 0.1664]))]


## Create LabeledPoints 



In [ ]:
from pyspark.mllib.regression import LabeledPoint

# create labelled points of vector size N out of an RDD with normalised (filename, td.idf-vector) items
def makeLabeledPoints(fn_vec_RDD): # RDD and N needed 
    # determine the true class as encoded in the filename and represent as 1 (spam) or 0 (good) 
    cls_vec_RDD = fn_vec_RDD.map(lambda v: (1 if v[0].startswith("spmsg") else 0, v[1])) # use a conditional expression to get the class label (True or False)
    # now create the LabeledPoint objects with (class, vector) arguments
    lp_RDD = cls_vec_RDD.map(lambda cls_vec: LabeledPoint(cls_vec[0], cls_vec[1]) ) 
    return lp_RDD 

# for testing
testLpRDD = makeLabeledPoints(rdd3)
print(testLpRDD.take(1))


[LabeledPoint(0.0, [0.0,0.07199034703813956,0.3016523635774738,0.0,0.0,0.0,0.17751412657278054,0.14398069407627911,0.1384429750733453,0.16641949366198175])]


##  Complete the preprocessing 


In [ ]:
# now apply the preprocessing chain to the data loaded 
# N is for controlling the vector size
def preprocess(rawRDD, N):
    """ take a (filename,text) RDD and transform into LabelledPoint objects 
        with class labels and a TF.IDF vector with N dimensions. 
    """
    a = prepareTokenRDD(rawRDD)
    b = normTFIDF(a , N)
    c = makeLabeledPoints(b)
    return c # apply tasks 2, 3 and 4 here
    

# and with this we can start the whole process from a directory, N is again the vector size
def loadAndPreprocess(directory, N):
    """ load lingspam data from a directory and create a training and test set of preprocessed data """
    trainRDD_testRDD = makeTestTrainRDDs(directory) # read from the directory using the function created in task 1
    (trainRDD, testRDD) =  trainRDD_testRDD# unpack the returned tuple
    return (preprocess(trainRDD, N), preprocess(testRDD, N)) # apply the preprocessing function defined above

trainLpRDD = preprocess(trainRDD, testDim) # prepare the training data
print(trainLpRDD.take(1)) # should look similar to previous cell's output

train_test_LpRDD = loadAndPreprocess('lemm', 100) # re-run with another vector size
(trainLpRDD, testLpRDD) = train_test_LpRDD
print(testLpRDD.take(1))
print(trainLpRDD.take(1))

[LabeledPoint(0.0, [0.20078443042124564,0.144815703802104,0.13139074894555822,0.0,0.0,0.07041005093669801,0.13403686030341527,0.13282429900512271,0.10525380815431767,0.08048409843153845])]
[PosixPath('lemm/part10'), PosixPath('lemm/part9'), PosixPath('lemm/part8'), PosixPath('lemm/part7'), PosixPath('lemm/part6'), PosixPath('lemm/part5'), PosixPath('lemm/part4'), PosixPath('lemm/part3'), PosixPath('lemm/part2'), PosixPath('lemm/part1')]
creating RDDs
/content/drive/My Drive/BigData2020/data/lingspam_public/lemm/part10
/content/drive/My Drive/BigData2020/data/lingspam_public/lemm/part9
/content/drive/My Drive/BigData2020/data/lingspam_public/lemm/part8
/content/drive/My Drive/BigData2020/data/lingspam_public/lemm/part7
/content/drive/My Drive/BigData2020/data/lingspam_public/lemm/part6
/content/drive/My Drive/BigData2020/data/lingspam_public/lemm/part5
/content/drive/My Drive/BigData2020/data/lingspam_public/lemm/part4
/content/drive/My Drive/BigData2020/data/lingspam_public/lemm/part3


## Train some classifiers  

In [ ]:
from pyspark.mllib.classification import NaiveBayes, LogisticRegressionWithLBFGS, SVMWithSGD
from pyspark import StorageLevel

# train the model with a LabeledPoint RDD.
def trainModel(lpRDD):
    """ Train 3 classifier models on the given RDD with LabeledPoint objects. A list of trained model is returned. """
    # Train a classifier model.
    print('Starting to train the model') # give some immediate feedback
    model1 = LogisticRegressionWithLBFGS.train(lpRDD) # this is the best model
    print('Trained LR (model1)')
    #print('type(model1)')
    model2 = NaiveBayes.train(lpRDD) # doesn't work well
    print('Trained NB (model2)')
    #print(type(model2))
    model3 = SVMWithSGD.train(lpRDD) # or this ...
    print('Trained SVM (model3)')
    return [model1, model2, model3]

def testModel(model, lpRDD):
    """ Tests the classification accuracy of the given model on the given RDD with LabeledPoint objects. """
    lpRDD.persist(StorageLevel.MEMORY_ONLY)
    # Make prediction and evaluate training set accuracy
    # Get the prediction and the ground truth label
    predictionAndLabel = lpRDD.map(lambda p: (model.predict(p.features), p.label)) # get the prediction and ground truth (label) for each item
    correct = predictionAndLabel.filter(lambda xv: xv[0] == xv[1]).count() # count the correct predictions 
    accuracy = correct/lpRDD.count() # and calculate the accuracy 
    print('Accuracy {:.1%} (data items: {}, correct: {})'.format(accuracy, lpRDD.count(), correct)) # report to console
    return accuracy # and return the value  

models = trainModel(trainLpRDD) # just for testing
testModel(models[2], trainLpRDD) # just for testing

Starting to train the model
Trained LR (model1)
Trained NB (model2)
Trained SVM (model3)
Accuracy 83.4% (data items: 2604, correct: 2171)


0.8337173579109063

## Automate training and testing


In [ ]:
# this function combines the previous two functions
# this method should take RDDs with LabeledPoints
def trainTestModel(trainRDD, testRDD):
    """ Trains 3 models and tests them on training and test data. Returns a matrix with the training and testing (rows) accuracy values for all models (columns). """
    models = trainModel(trainRDD) # train models on the training set
    results = [[], []] # matrix for 2 modes (training/test) vs n models (currently 3)
    for mdl in models:
        print('Training')
        results[0].append(mdl) # test the model on the training set
        print('Testing')
        results[1].append(testModel(mdl , testRDD)) # test the model on the test set
    return results

def trainTestFolder(folder,N):
    """ Reads data from a folder, preproceses the data, and trains and evaluates models on it. """
    print('Start loading and preprocessing') 
    train_test_LpRDD = loadAndPreprocess(folder,N) # create the RDDs
    print('Finished loading and preprocessing')
    (trainLpRDD, testLpRDD) = train_test_LpRDD # unpack the RDDs 
    return trainTestModel(trainLpRDD,testLpRDD) # train and test

trainTestFolder('lemm', 1000) 

## Run experiments 


In [ ]:
folder = 'bare'
vectorsizes = [3, 30, 300, 3000, 30000]
print('EXPERIMENT 1: Testing different vector sizes')
results_vectorsizes = []
for n in vectorsizes:
    print('\nN = {}'.format(n))
    result = {'n': n, 't': folder}
    result['acc'] = trainTestFolder(folder, n)
    results_vectorsizes.append(result)

EXPERIMENT 1: Testing different vector sizes

N = 3
Start loading and preprocessing
[PosixPath('bare/part10'), PosixPath('bare/part9'), PosixPath('bare/part8'), PosixPath('bare/part7'), PosixPath('bare/part6'), PosixPath('bare/part5'), PosixPath('bare/part4'), PosixPath('bare/part3'), PosixPath('bare/part2'), PosixPath('bare/part1')]
creating RDDs
/content/drive/My Drive/BigData2020/data/lingspam_public/bare/part10
/content/drive/My Drive/BigData2020/data/lingspam_public/bare/part9
/content/drive/My Drive/BigData2020/data/lingspam_public/bare/part8
/content/drive/My Drive/BigData2020/data/lingspam_public/bare/part7
/content/drive/My Drive/BigData2020/data/lingspam_public/bare/part6
/content/drive/My Drive/BigData2020/data/lingspam_public/bare/part5
/content/drive/My Drive/BigData2020/data/lingspam_public/bare/part4
/content/drive/My Drive/BigData2020/data/lingspam_public/bare/part3
/content/drive/My Drive/BigData2020/data/lingspam_public/bare/part2
/content/drive/My Drive/BigData2020/d

KeyboardInterrupt: ignored

In [ ]:
n = 3000
typeFolders = ['bare', 'stop', 'lemm', 'lemm_stop']
print('EXPERIMENT 2: Testing different data types')
results_preprocessing = []
for folder in typeFolders:
    print('\nPath = {}'.format(folder))
    result = {'n': n, 't': folder}
    result['acc'] = trainTestFolder(folder, n)
    results_preprocessing.append(result) 